In [1]:
import numpy as np
import os
from glob import glob
import pandas as pd
import matplotlib.pyplot as plt

# Hjælpefunktioner

In [2]:
def biniaryyn_variable(dataframe, variable):
       
    # binary labels
    dataframe[variable] = dataframe[variable].replace(['.: Missing Form/Incomplete Workbook','0: No','1: Yes'],[0.5,0,1])

    
def pain_values(dataframe,variable):
    
    # Set missing values as mean        
    dataframe[variable] = dataframe[variable].replace(['0: None','1: Mild','2: Moderate','3: Severe','4: Extreme'],[0,1,2,3,4])
    
    mean = np.mean(dataframe[dataframe[variable]!='.: Missing Form/Incomplete Workbook'][variable])
    dataframe[variable] = dataframe[variable].replace(['.: Missing Form/Incomplete Workbook'],[mean])
    
    

def pain_values2(dataframe,variable):

    dataframe[variable] = dataframe[variable].replace(['0: No pain', '1: 1', '2: 2', '3: 3', '4: 4', '5: 5','6: 6', '7: 7', '8: 8', '9: 9','10: Pain as bad as you can imagine'],[0,1,2,3,4,5,6,7,8,9,10])
    
    mean = np.mean(dataframe[dataframe[variable]!='.: Missing Form/Incomplete Workbook'][variable])
    dataframe[variable] = dataframe[variable].replace(['.: Missing Form/Incomplete Workbook'],[mean])
    
def emotional_values(dataframe, variable):
    # Replace missing values with mean 
    
    dataframe[variable] = dataframe[variable].replace(['1: Rarely/none of the time (<1 day)','2: Some of the time (1-2 days)','3: Much of the time (3-4 days)','4: Most or all of the time (5-7 days)'],[1,2,3,4])
    
    mean = np.mean(dataframe[dataframe[variable]!='.: Missing Form/Incomplete Workbook'][variable])
    
    dataframe[variable] = dataframe[variable].replace(['.: Missing Form/Incomplete Workbook'],[mean])  

    
def lifechane_values(dataframe, variable):
        dataframe[variable] = dataframe[variable].replace(['.: Missing Form/Incomplete Workbook','0: Not at all','1: Mildly', '2: Moderately',  '3: Severely', '4: Totally'],[0,0,1,2,3,4])

def nan_to_mean(dataframe, variable):
        mean = np.mean(dataframe[variable])
        dataframe[variable] = dataframe[variable].replace(['.: Missing Form/Incomplete Workbook'],[mean])

def vitamins(dataframe, variable):
            dataframe[variable] = dataframe[variable].replace(['.: Missing Form/Incomplete Workbook',
                                                                 '1: Didn t take',
                                                                 '2: A few days per month',
                                                                 '3: 4-6 days per week',
                                                                 '4: Every day',
                                                                 '5: 1-3 days per week'],[2,0,1,3,5,4])

# Get subjects

In [3]:
#subjects
idx = np.loadtxt("SUBJECTS_09-03.txt",dtype='int')
len(idx)

1121

In [4]:
idx

array([9011918, 9010952, 9020856, ..., 9992358, 9421302, 9429578])

### Enroll data 

In [5]:
enroll = "C:/Users/python_test/Desktop/Speciale/OAIdata21/OAIdata21/Enrollees.txt"
df_enroll = pd.read_csv(enroll, sep="|", header=0, index_col="ID")
df_enroll = df_enroll["P02SEX"].replace(['1: Male','2: Female'],[0,1])

In [6]:
visit00 = "C:/Users/python_test/Desktop/Speciale/OAIdata21/OAIdata21/AllClinical00.txt"
df_visit00 = pd.read_csv(visit00, sep="|", header=0, index_col="ID")

# choose which variables to use
    # V00AGE age
age = df_visit00["V00AGE"]

### Load dataset 

In [7]:
def add_visit(visit, subjects):
    path =  "C:/Users/python_test/Desktop/Speciale/OAIdata21/OAIdata21/AllClinical"+visit[1:]+".txt"

    df_visit = pd.read_csv(path, sep="|", header=0, index_col="ID")
    df_visit = df_visit[df_visit.index.isin(subjects)] 
    
    if visit in ["V00"]:
        variables_of_interest = [visit+"WOMTSL", visit+"WOMTSR",# visit+"PMLKRCV", visit+"PMRKRCV", # pain
                               visit+"DILKN2",visit+"DILKN10",visit+"DILKN11",visit+"DIRKN2",visit+"DIRKN10",visit+"DIRKN11", visit+"KQOL2", #limitations
                               visit+"PASE",
                               "V00BONEFX", #fracture after 45 month
                               "P01BMI"] #BMI] # Exercise
        
    
    if visit in ["V01","V03","V05","V06","V08"]:
        variables_of_interest = [visit+"WOMTSL", visit+"WOMTSR",# visit+"PMLKRCV", visit+"PMRKRCV", # pain
                               visit+"DILKN2",visit+"DILKN10",visit+"DILKN11",visit+"DIRKN2",visit+"DIRKN10",visit+"DIRKN11", visit+"KQOL2", #limitations
                               visit+"PASE",
                               visit+"BONFX", #fracture last month
                               visit+"BMI"] #BMI] # Exercise

        
    if visit in ["V02","V04","V07","V09"]:
        variables_of_interest = [visit+"WOMTSL", visit+"WOMTSR", # pain
                               visit+"DILKN2",visit+"DILKN10",visit+"DILKN11",visit+"DIRKN2",
                               visit+"DIRKN10",visit+"DIRKN11", visit+"KQOL2"] #limitations # Exercise
    
    df_visit = df_visit[variables_of_interest]

    if visit in ["V01","V03","V05","V06","V08"]:
        biniaryyn_variable(df_visit,visit+"BONFX")
        
    if visit in ["V00"]:
        biniaryyn_variable(df_visit,"V00BONEFX")

        #pain_values2(df_visit,visit+"PMRKRCV")
        #pain_values2(df_visit,visit+"PMLKRCV")
    #biniaryyn_variable

    pain_values(df_visit,visit+"DILKN10")
    pain_values(df_visit,visit+"DILKN11")
    pain_values(df_visit,visit+"DILKN2")
    pain_values(df_visit,visit+"DIRKN10")
    pain_values(df_visit,visit+"DIRKN11")
    pain_values(df_visit,visit+"DIRKN2")

    lifechane_values(df_visit,visit+"KQOL2")

    #Set missing values to mean
    for col in df_visit.columns:
        mean = np.mean(df_visit[col])
        df_visit[col] = df_visit[col].fillna(mean)

    return df_visit

In [8]:
v0 = add_visit("V00", idx)
v1 = add_visit("V01", idx)
v2 = add_visit("V02", idx)
v3 = add_visit("V03", idx)
v4 = add_visit("V04", idx)
v5 = add_visit("V05", idx)
v6 = add_visit("V06", idx)
v7 = add_visit("V07", idx)
v8 = add_visit("V08", idx)
#v9 = add_visit("V09", idx)
#['V09BONFX', 'V09BMI', 'V09PASE'] 

In [9]:
#Interpolate
v2["V02PASE"] = v1["V01PASE"]+ (v3["V03PASE"]-v1["V01PASE"])/2
v2["V02BONFX"] = (v1["V01BONFX"]+v3["V03BONFX"])/2
v2["V02BMI"] = v1["V01BMI"]+(v3["V03BMI"]-v1["V01BMI"])/2

v4["V04PASE"] = v3["V03PASE"]+ (v5["V05PASE"]-v3["V03PASE"])/2
v4["V04BONFX"] = (v5["V05BONFX"]+v3["V03BONFX"])/2
v4["V04BMI"] = v3["V03BMI"]+(v5["V05BMI"]-v3["V03BMI"])/2

v7["V07PASE"] = v6["V06PASE"]+ (v8["V08PASE"]-v6["V06PASE"])/2
v7["V07BONFX"] = (v8["V08BONFX"]+v6["V06BONFX"])/2
v7["V07BMI"] = v6["V06BMI"]+(v8["V08BMI"]-v6["V06BMI"])/2

### Get labels

In [10]:
#labels
label_path = "C:/Users/python_test/Desktop/Speciale/OAIdata21/OAIdata21/Outcomes99.txt"
df_labels = pd.read_csv(label_path, sep="|", header=0, index_col="id")
set(df_labels[df_labels["V99ERKVSPR"]!=".: Missing Form/Incomplete Workbook"]["V99ERKVSPR"])

{'0: Baseline',
 '10: 96-month',
 '1: 12-month',
 '2: 18-month',
 '3: 24-month',
 '4: 30-month',
 '5: 36-month',
 '6: 48-month',
 '7: 60-month',
 '8: 72-month',
 '9: 84-month'}

In [11]:
a=np.loadtxt("SUBJECTS_09-03.txt",dtype='str')
#[v0,v1,v3,v5,v6]
l_v356 = []
kontrol = []
l_v135 = []
l_v013 = []
count = 0
l=0
#V99ELKTPPR Outcomes: left knee, type of partial follow- up knee replacement 
#V99ELKVSPR Outcomes: left knee, closest OAI contact prior to follow- up knee replacement


for id in df_labels[df_labels.index.isin(a.astype(int))].index:
    if df_labels.loc[id]["V99ELKTLPR"]== '.: Missing Form/Incomplete Workbook':
        kontrol.append(id)
print(len(kontrol))


for id in df_labels[df_labels.index.isin(a.astype(int))].index:
    if df_labels.loc[id]["V99ELKTLPR"] in ['1: Total', '2: Partial']:
    
        if df_labels.loc[id]["V99ELKVSPR"] in ["6: 48-month","7: 60-month","8: 72-month","9: 84-month","10: 96-month"]:
            l_v356.append(id)
            l_v356.append(kontrol[0])
            l_v356.append(kontrol[1])
            kontrol = kontrol[2:]

        elif df_labels.loc[id]["V99ELKVSPR"] in ['5: 36-month']:
            l_v135.append(id)
            l_v135.append(kontrol[0])
            l_v135.append(kontrol[1])
            kontrol = kontrol[2:]

        elif df_labels.loc[id]["V99ELKVSPR"] in ['3: 24-month', '4: 30-month']:
            l_v013.append(id)
            l_v013.append(kontrol[0])
            l_v013.append(kontrol[1])
            kontrol = kontrol[2:]

        elif df_labels.loc[id]["V99ELKVSPR"] in ['0: Baseline', '1: 12-month', '2: 18-month']:
            kontrol = kontrol[2:]
            count +=1

    if df_labels.loc[id]["V99ERKTLPR"] in ['1: Total', '2: Partial']:

            if df_labels.loc[id]["V99ERKVSPR"] in ["6: 48-month","7: 60-month","8: 72-month","9: 84-month","10: 96-month"]:
                l_v356.append(id)
                l_v356.append(kontrol[0])
                l_v356.append(kontrol[1])
                kontrol = kontrol[2:]

            elif df_labels.loc[id]["V99ERKVSPR"] in ['5: 36-month']:
                l_v135.append(id)
                l_v135.append(kontrol[0])
                l_v135.append(kontrol[1])
                kontrol = kontrol[2:]

            elif df_labels.loc[id]["V99ERKVSPR"] in ['3: 24-month', '4: 30-month']:
                l_v013.append(id)
                l_v013.append(kontrol[0])
                l_v013.append(kontrol[1])
                kontrol = kontrol[2:]

            elif df_labels.loc[id]["V99ERKVSPR"] in ['0: Baseline', '1: 12-month', '2: 18-month']:
                kontrol = kontrol[2:]
                count +=1

print(len(kontrol))
# 46 Deleted as they had knee surgery witin the before 24-month visit

865
123


In [12]:
#l_v356.to_csv("l_v356")
#l_v135.to_csv("l_v135")
#l_v013.to_csv("l_v013")

np.savetxt("l_v013.csv", 
           l_v013,
           delimiter =", ", 
           fmt ='% s')

np.savetxt("l_v135.csv", 
           l_v135,
           delimiter =", ", 
           fmt ='% s')

np.savetxt("l_v356.csv", 
           l_v356,
           delimiter =", ", 
           fmt ='% s')

# Make data set with multiple visits

In [13]:
#Make data set with multiple visits
v01356 = pd.concat( [df_enroll,age,v0,v1,v3,v5,v6, df_labels["V99ELKVSPR"], df_labels["V99ERKVSPR"]], axis=1, join="inner")
v01356 = v01356.loc[l_v356]
v01356["V99ELKVSPR"]=np.where(v01356["V99ELKVSPR"]==".: Missing Form/Incomplete Workbook",0,1)
v01356["V99ERKVSPR"]=np.where(v01356["V99ERKVSPR"]==".: Missing Form/Incomplete Workbook",0,1)

v01356 = v01356.dropna()
v01356.to_csv("v01356.csv")

# Make dataset with three visits

In [14]:
v013 = pd.concat( [df_enroll,age,v0,v1,v3,df_labels["V99ELKVSPR"], df_labels["V99ERKVSPR"]], axis=1, join="inner")
v013 = v013.loc[l_v013]

v135 = pd.concat( [df_enroll,age,v1,v3,v5,df_labels["V99ELKVSPR"], df_labels["V99ERKVSPR"]], axis=1, join="inner")
v135 = v135.loc[l_v135]
v135.set_axis(list(v013.columns), axis=1, inplace=True)

v356 = pd.concat( [df_enroll,age,v3,v5,v6,df_labels["V99ELKVSPR"], df_labels["V99ERKVSPR"]], axis=1, join="inner")
v356 = v356.loc[l_v356]
v356.set_axis(list(v013.columns), axis=1, inplace=True)

In [15]:
v_all = pd.concat([v013,v135,v356])
v_all["V99ELKTPPR"]=np.where(v_all["V99ELKVSPR"]==".: Missing Form/Incomplete Workbook",0,1)
v_all["V99ERKTPPR"]=np.where(v_all["V99ERKVSPR"]==".: Missing Form/Incomplete Workbook",0,1)

In [16]:
from sklearn.model_selection import train_test_split
matched_15032022, validation_15032022 = train_test_split(v_all, test_size=0.20, random_state=42, shuffle=True)
matched_15032022.to_csv("matched!_15032022")
validation_15032022.to_csv("validation!_15032022")

In [17]:
(v356.index)

Int64Index([9014797, 9001897, 9002316, 9014883, 9002817, 9003126, 9030718,
            9003815, 9003895, 9034451,
            ...
            9360202, 9992318, 9375122, 9375317, 9992358, 9376106, 9379276,
            9992358, 9381278, 9382719],
           dtype='int64', length=738)

## Check if images there

In [18]:
directory = 'Z:/OsteoarthritisInitiative/my_predictions/nii_files'
segments_in_folder = os.listdir(directory)

print("initial amount l_v013", len(l_v013))
mis013 = []
n_imgs = 0
for sub in v013.index:
    for v in ["V00","V01","V03"]:
        left = str(sub)+"-Left-"+v+"_PRED.nii.gz"
        right = str(sub)+"-Right-"+v+"_PRED.nii.gz"
        
        if left in segments_in_folder:
            n_imgs+=1
        
        if right in segments_in_folder:
            n_imgs+=1
            
        else:
            mis013.append(sub)
        
print("In folder l_v013: ", n_imgs)
print("missing l_v013: ", len(mis013))


print("initial amount v135", len(v135))
mis135 = []
n_imgs = 0
for sub in v135.index:
    for v in ["V01","V03","V05"]:
        left = str(sub)+"-Left-"+v+"_PRED.nii.gz"
        right = str(sub)+"-Right-"+v+"_PRED.nii.gz"
        
        if left in segments_in_folder:
            n_imgs+=1
        
        if right in segments_in_folder:
            n_imgs+=1
            
        else:
            mis135.append(sub)
        
print("In folder v135: ", n_imgs)
print("missing v135: ", len(mis135))


print("initial amount v356", len(v356))
mis356 = []
n_imgs = 0
for sub in v356.index:
    for v in ["V03","V05","V06"]:
        left = str(sub)+"-Left-"+v+"_PRED.nii.gz"
        right = str(sub)+"-Right-"+v+"_PRED.nii.gz"
        
        if left in segments_in_folder:
            n_imgs+=1
        
        elif right in segments_in_folder:
            n_imgs+=1
            
        else:
            mis356.append(sub)
        
print("In folder v356: ", n_imgs)
print("missing v356: ", len(mis356))

initial amount l_v013 123
In folder l_v013:  678
missing l_v013:  25
initial amount v135 114
In folder v135:  583
missing v135:  26
initial amount v356 738
In folder v356:  1989
missing v356:  225


In [19]:
#130+82+482
#126+81+468
#83+61+327
#79+58+317
#45+26+282
25+26+225

276

In [20]:
len(set(mis356+mis135+mis013))

158

### get missing images

In [21]:
a=np.loadtxt("SUBJECTS_09-03.txt",dtype='str')
all_img = []

for subject in a:
    #9000798-Left-V00
    
    for v in ["V00","V01","V03","V05","V06"]:
        img_name = subject +"-Left-"+v        
        all_img.append( img_name )
        img_name2 = subject +"-Right-"+v
        all_img.append( img_name2 )

In [22]:
directory = 'Z:/OsteoarthritisInitiative/NIFTY'
b = os.listdir(directory)
b = [filename[:-7] for filename in b]

In [23]:
print("before: ", len(all_img))

missing = []
for i in all_img:
    if i in b:
        #print("in b")
        continue
        
    
    if i not in b:
        #print(i)
        missing.append(i+".nii.gz")

#all_img
print("Missing: ", len(missing))

before:  11210
Missing:  1756


In [24]:
#pd.DataFrame(missing, index=False)
np.savetxt("missing.csv", 
           missing,
           delimiter =", ", 
           fmt ='% s')

Check mri complete

In [25]:
subs = [int(file[:7]) for file in missing]

df_path = "C:/Users/python_test/Desktop/Speciale/OAIdata21/OAIdata21/MRI03.txt"
df_mri_1 = pd.read_csv(df_path, sep="|", header=0, index_col="ID")
#set(df_labels[df_labels["V99ERKVSPR"]!=".: Missing Form/Incomplete Workbook"]["V99ERKVSPR"])

In [26]:
for i in missing:
    if i[:7] in df_mri_1[df_mri_1["V03MNDREAS"]=='.: Missing Form/Incomplete Workbook'].index:
        print("wuhuu")

In [27]:
df_mri_1

#missing

,VERSION,V03MEXAMTP,V03MNDREAS,V03MRBARCD,V03MRCOMP,V03MRDATE,V03MRMARK,V03MRSIDE,V03MRSURDY,V03MRTECID,V03QCRESLT,V03SCNUPGR
ID,,,,,,,,,,,,
9000099,3.2.3,L COR T1 3D FLASH WE,8: Knee with short protocol,NaN,0: No,NaN,.: Missing Form/Incomplete Workbook,2: Left,NaN,NaN,NaN,.: Missing Form/Incomplete Workbook
9000099,3.2.3,L SAG T2 MAP,8: Knee with short protocol,NaN,0: No,NaN,.: Missing Form/Incomplete Workbook,2: Left,NaN,NaN,NaN,.: Missing Form/Incomplete Workbook
9000099,3.2.3,THIGH T1,.: Missing Form/Incomplete Workbook,1.661191e+10,1: Yes,08/10/2007,.: Missing Form/Incomplete Workbook,3: Bilateral,NaN,B056,'YD': Not QCd and accepted by default,0: Original OAI Siemens Trio MR used
9000099,3.2.3,OAI Prescription,.: Missing Form/Incomplete Workbook,1.661191e+10,1: Yes,08/10/2007,.: Missing Form/Incomplete Workbook,3: Bilateral,NaN,B056,'YD': Not QCd and accepted by default,0: Original OAI Siemens Trio MR used
9000099,3.2.3,L SAG 3D DESS WE,.: Missing Form/Incomplete Workbook,1.661191e+10,1: Yes,08/10/2007,0: No,2: Left,NaN,B056,'YD': Not QCd and accepted by default,0: Original OAI Siemens Trio MR used
...,...,...,...,...,...,...,...,...,...,...,...,...
9999878,3.2.3,R SAG IW TSE FS,.: Missing Form/Incomplete Workbook,1.661141e+10,1: Yes,01/03/2007,0: No,1: Right,124.0,C016,'YD': Not QCd and accepted by default,0: Original OAI Siemens Trio MR used
9999878,3.2.3,R AXIAL MPR,.: Missing Form/Incomplete Workbook,1.661141e+10,1: Yes,01/03/2007,0: No,1: Right,124.0,C016,'YD': Not QCd and accepted by default,0: Original OAI Siemens Trio MR used
9999878,3.2.3,R COR MPR,.: Missing Form/Incomplete Workbook,1.661141e+10,1: Yes,01/03/2007,0: No,1: Right,124.0,C016,'YD': Not QCd and accepted by default,0: Original OAI Siemens Trio MR used
